In this tutorialwe will create a pipeline that uses an input LSL stream.

In [ ]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [ ]:
import mindpype as mp
import numpy as np
# This test requires a running LSL stream with the following properties:
# - Type: "EEG"

# This test also requires a marker stream with the following properties:
# - Type: "Markers"
# - Marker Format: ".*flash.*"

First we will specify the channel map and selected channels.

In [ ]:
channels = [i for i in range(3,17)]

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline. After creating the session we will create a graph which represents the pipeline.

In [ ]:
# Create a session and graph
session = mp.Session.create()
graph = mp.Graph.create(session)

Next, we will create an LSL input stream using the ```create_marker_coupled_data_stream()``` factory method. We will pass in the session, the stream predicate, the channels, the time offset, and the marker format to create the LSL object.

Then, we will create a tensor from the LSL input stream to serve as one input to our addition node. The LSL stream is volatile so to create the tensor, we will use the ```create_from_handle()``` factory method and pass in our LSL input stream object that we just created. 

We will create a second input tensor using the ```create_from_data()``` factory method.

We will also create a tensor to store our output data. For the output tensor, the library will automatically determine the correct size for the tensor, so we can create a "virtual" tensor, meaning that it can be modified by the library and does not contain any user provided data.

In [ ]:
# Create an LSLInputStream using the factory method
# To create the object, we must pass the session, the stream predicate, the channels, the time offset, and the marker format
# Alternatively, we can pass the data and marker stream infos, which are pylsl.StreamInfo objects

lsl_object = mp.source.InputLSLStream.create_marker_coupled_data_stream(
    session, "type='EEG'",
    channels, marker_fmt=".*flash.*", Ns=1000
    )


# Create a tensor from the LSLInputStream. Since the LSL stream is volatile (not static) between trials,
# we create the tensor from "handle", passing the stream object and the shape of the tensor
t_in = mp.Tensor.create_from_handle(session, (len(channels), 1000), lsl_object)

# Create an output tensor
t_out = mp.Tensor.create(session, t_in.shape)

Next, we will add a filter node to the graph using the ```add_to_graph()``` factory method.

In [ ]:
# Add an addition node to the graph, passing the graph, the input tensors, and the output tensor
order = 4
bandpass = (8,35) # in Hz
fs = 250
filter_obj = mp.Filter.create_butter(session,order,bandpass,btype='bandpass',fs=fs,implementation='sos')

mp.kernels.FilterKernel.add_to_graph(graph, t_in, filter_obj, t_out)

At this point we have created the processing pipeline, and added all of the required components. We can now use a graph method to verify that the structure of the pipeline is valid.

Following the verification of the pipeline, we should now initialize the graph. This step is required for pipelines that have methods that need to be trained or fit (ie. classifiers), but optional for other pipelines.


In [ ]:
# Verify and initialize the graph
graph.verify()
graph.initialize()

Finally, we will execute the graph 10 times, printing each sum output.

In [ ]:
import matplotlib.pyplot as plt
def plot_trial(X):
    fig, ax = plt.subplots()
    #ax.clear()
    t = np.arange(0, X.shape[1]/128, 1/128)
    for i_ch, ch in enumerate(range(X.shape[0])):
        ax.plot(t, X[i_ch,:]+i_ch*200, label=ch)
    ax.set_yticks(())
    ax.set_xlabel('Time (s)', fontsize=28)
    plt.show()


In [ ]:

for i in range(10):
    # Execute the graph. The execute method will automatically wait for the correct marker
    sts = graph.execute()
    plot_trial(t_out.data)
    
